## DNN model for MNIST dataset

### Prepare Models

In [1]:
import numpy as np
import torch
import tensorflow as tf

device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-01-31 22:47:13.090089: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 22:47:13.122303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 22:47:13.122331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 22:47:13.123021: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 22:47:13.127618: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

# Convert to PyTorch format [batch_size, total pixels]
# Since images are already normalized and flattened for TensorFlow, we can use the same arrays
train_images_pt = torch.tensor(train_images_tf).float()
test_images_pt = torch.tensor(test_images_tf).float()
train_labels_pt = torch.tensor(train_labels)
test_labels_pt = torch.tensor(test_labels)

In [3]:
from tensorflow import keras
num_classes = 10

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
    keras.layers.Dense(56, activation='relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

2024-01-31 22:47:58.694358: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 22:47:58.695264: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-31 22:47:58.695357: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=20, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/20


2024-01-31 22:48:14.231128: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-31 22:48:14.283043: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f442833f970 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-31 22:48:14.283061: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-01-31 22:48:14.286196: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-31 22:48:14.294315: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1706712494.340829  528429 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1688/1688 [==============================] - 3s 1ms/step - loss: 0.3180 - accuracy: 0.9109 - val_loss: 0.1517 - val_accuracy: 0.9593
Epoch 2/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1557 - accuracy: 0.9554 - val_loss: 0.1089 - val_accuracy: 0.9682
Epoch 3/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1157 - accuracy: 0.9666 - val_loss: 0.0974 - val_accuracy: 0.9710
Epoch 4/20
1688/1688 [==============================] - 2s 892us/step - loss: 0.0914 - accuracy: 0.9726 - val_loss: 0.0902 - val_accuracy: 0.9753
Epoch 5/20
1688/1688 [==============================] - 2s 992us/step - loss: 0.0765 - accuracy: 0.9770 - val_loss: 0.0873 - val_accuracy: 0.9755
Epoch 6/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0639 - accuracy: 0.9806 - val_loss: 0.0992 - val_accuracy: 0.9705
Epoch 7/20
1688/1688 [==============================] - 2s 991us/step - loss: 0.0553 - accuracy: 0.9838 - val_loss: 0.0892 - val_accuracy: 0.97

#### Convert to Pytorch DNN model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, num_classes=10):
        super(Net, self).__init__()
        # Fully connected layers / Dense block
        # First dense layer
        self.fc1 = nn.Linear(28*28, 56)  # Flatten 28*28 and feed into 56 neurons
        
        # Second dense layer (output layer)
        self.fc2 = nn.Linear(56, num_classes)  # 56 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Fully connected layers with ReLU activation for the first layer
        x = F.relu(self.fc1(x))

        # Output layer with no activation
        # Softmax will be applied externally during training and evaluation
        x = self.fc2(x)
        return F.log_softmax(x, dim = 1)
    
model_pt = Net()

In [6]:
# Transfer weights for the first dense layer (fc1) from model_tf to model_pt
weights, biases = model_tf.layers[0].get_weights()
model_pt.fc1.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc1.bias = nn.Parameter(torch.from_numpy(biases))

# Transfer weights for the second dense layer (fc2) from model_tf to model_pt
weights, biases = model_tf.layers[1].get_weights()
model_pt.fc2.weight = nn.Parameter(torch.from_numpy(np.transpose(weights, (1, 0))))
model_pt.fc2.bias = nn.Parameter(torch.from_numpy(biases))

In [7]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images[38].reshape(1, 28*28)  # Reshape to (1, 784) for DNN
controlled_input_pt = torch.from_numpy(controlled_input_tf).float()

# Test TensorFlow Model
output_tf = model_tf.predict(controlled_input_tf) 
print("TensorFlow Basic Model Output:", output_tf)

# Test PyTorch Model
model_pt.eval()  # Set PyTorch model to evaluation mode
with torch.no_grad():
    output_pt = model_pt(controlled_input_pt)
print("PyTorch Basic Model Output:", torch.exp(output_pt).numpy())

1/1 [==============================] - 0s 33ms/step
TensorFlow Basic Model Output: [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
PyTorch Basic Model Output: [[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]


In [8]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDataset for test data
test_dataset = TensorDataset(test_images_pt, test_labels_pt)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

def evaluate_pytorch_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

# Evaluate the PyTorch model
accuracy = evaluate_pytorch_model(model_pt, test_loader)
print(f'Accuracy of the PyTorch model on the test images: {accuracy:.8f}%')


Accuracy of the PyTorch model on the test images: 97.38000000%


### Test Converted Pytorch Model

In [9]:
def get_predictions_tf(model, test_images, batch_size=32):
    predictions = []
    for i in range(0, len(test_images), batch_size):
        batch = test_images[i:i+batch_size]
        pred = model.predict(batch)
        predictions.extend(np.argmax(pred, axis=1))
    return predictions

def get_predictions_pt(model, test_images, batch_size=32):
    model.eval()
    predictions = []
    with torch.no_grad():
        for i in range(0, len(test_images), batch_size):
            batch = test_images[i:i+batch_size]
            pred = model(batch)
            predictions.extend(torch.argmax(pred, axis=1).tolist())
    return predictions

In [10]:
# Generate predictions
predictions_tf = get_predictions_tf(model_tf, test_images_tf)
predictions_pt = get_predictions_pt(model_pt, test_images_pt)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/1 [==============================] - 0s 23ms/step
Number of mismatches: 0 out of 10000 samples


#### Save the Model

In [12]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open('input-dense-dense_784_56_10.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpsdalsyva/assets


INFO:tensorflow:Assets written to: /tmp/tmpsdalsyva/assets
2024-01-31 23:00:06.103994: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-31 23:00:06.104008: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-31 23:00:06.104174: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpsdalsyva
2024-01-31 23:00:06.104505: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-31 23:00:06.104510: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpsdalsyva
2024-01-31 23:00:06.105356: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-31 23:00:06.105667: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-01-31 23:00:06.122705: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [11]:
# Save entire model
torch.save(model_pt, "input-dense-dense_784_56_10.pt")

# Save only the state_dict
torch.save(model_pt.state_dict(), "input-dense-dense_784_56_10.pth")

In [15]:
test_images_pt[0].shape

torch.Size([784])

In [16]:
torch.onnx.export(model_pt, controlled_input_pt, "input-dense-dense_784_56_10.onnx")

### Orion Specialized q_aware_model


In [17]:
import tensorflow_model_optimization as tfmot

# Apply quantization to the layers
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model_tf)

# 'quantize_model' requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 784)               3         
 yer)                                                            
                                                                 
 quant_dense (QuantizeWrapp  (None, 56)                43965     
 erV2)                                                           
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                575       
 pperV2)                                                         
                                                                 
Total params: 44543 (174.00 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 13 (52.00 Byte)
_________________________________________________________________


In [26]:
batch_size = 32
epochs = 20
history = q_aware_model.fit(train_images_tf, train_labels,
                            epochs=epochs,
                            validation_split=0.1)

# Evaluate the model
test_loss, test_acc = q_aware_model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/20
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0030 - accuracy: 0.9989 - val_loss: 0.1957 - val_accuracy: 0.9760
Epoch 2/20
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0044 - accuracy: 0.9984 - val_loss: 0.1857 - val_accuracy: 0.9775
Epoch 3/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0033 - accuracy: 0.9990 - val_loss: 0.2262 - val_accuracy: 0.9707
Epoch 4/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0048 - accuracy: 0.9985 - val_loss: 0.1971 - val_accuracy: 0.9758
Epoch 5/20
1688/1688 [==============================] - 3s 2ms/step - loss: 0.0022 - accuracy: 0.9993 - val_loss: 0.1968 - val_accuracy: 0.9742
Epoch 6/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0039 - accuracy: 0.9987 - val_loss: 0.2086 - val_accuracy: 0.9748
Epoch 7/20
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0014 - accuracy: 0.9997 - val_loss: 0.1972 - val_accuracy:

In [29]:
# Generate predictions
q_predictions_tf = get_predictions_tf(q_aware_model, test_images_tf)

# Compare predictions
mismatches = sum(p1 != p2 for p1, p2 in zip(q_predictions_tf, predictions_pt))
print(f"Number of mismatches: {mismatches} out of {len(test_images_tf)} samples")


1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 14ms/step
Number of mismatches: 166 out of 10000 samples


In [32]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [np.array(train_images[i:i+1], dtype=np.float32)]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
q_aware_tflite_model = converter.convert()

# Save the model to disk
open("q_input-dense-dense_784_56_10.tflite", "wb").write(q_aware_tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpt1ux3e0e/assets


INFO:tensorflow:Assets written to: /tmp/tmpt1ux3e0e/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-01-31 23:24:20.362242: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-31 23:24:20.362256: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-31 23:24:20.362351: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpt1ux3e0e
2024-01-31 23:24:20.362995: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-31 23:24:20.363002: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpt1ux3e0e
2024-01-31 23:24:20.364794: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
20

46896

In [16]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# The input needs to be quantized, so we retrieve the quantization parameters
input_scale, input_zero_point = input_details[0]['quantization']
output_scale, output_zero_point = output_details[0]['quantization']

# Normalize and quantize the test images
test_images_quant = (test_images / input_scale + input_zero_point).astype(np.int8)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [17]:
# Evaluate the quantized TFLite model
correct_predictions = 0
for i in range(len(test_images)):
    test_image = np.expand_dims(test_images_quant[i], axis=0)
    
    # Set the value for the input tensor
    interpreter.set_tensor(input_details[0]['index'], test_image)
    
    # Run the inference
    interpreter.invoke()

    # Retrieve the output and dequantize
    output = interpreter.get_tensor(output_details[0]['index'])
    output = np.argmax(output, axis=1)
    predicted_class = output[0]
    if predicted_class == test_labels[i]:
        correct_predictions += 1

# Calculate the accuracy
accuracy = correct_predictions / len(test_images) * 100
print(f'Quantized model accuracy: {accuracy:.2f}%')

ValueError: Cannot set tensor: Dimension mismatch. Got 3 but expected 2 for input 0.

: 

In [5]:
!scarb new orion

Created `orion` package.


In [11]:
import numpy as np
import tensorflow as tf
import os


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

# Create an object with all tensors (an input + all weights and biases)
tensors = {
    "fc1_weights": interpreter.get_tensor(1), 
    "fc1_bias": interpreter.get_tensor(2), 
    "fc2_weights": interpreter.get_tensor(4), 
    "fc2_bias": interpreter.get_tensor(5)
}

# Create the directory if it doesn't exist
os.makedirs('./orion_dnn/src/generated', exist_ok=True)


In [20]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

input_cairo = generate_input_cairo(controlled_input_tf[0])

with open("orion_dnn/src/generated/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)

In [12]:

for tensor_name, tensor in tensors.items():
    with open(os.path.join('./orion_dnn/src', 'generated', f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use array::{ArrayTrait, SpanTrait};\n" +
            "use orion::operators::tensor::{core::{Tensor, TensorTrait}};\n" +
            "use orion::operators::tensor::FP16x16Tensor;\n" +
            "use orion::numbers::fixed_point::implementations::fp16x16::core::{FP16x16, FixedTrait};\n" +
            "\n" + f"fn {tensor_name}() -> Tensor<FP16x16>" + "{\n\n" + 
            "let mut shape = ArrayTrait::new();\n"
        )
        for dim in tensor.shape:
            f.write(f"shape.append({dim});\n")
        f.write("let mut data = ArrayTrait::new();\n")

        for val in np.nditer(tensor.flatten()):
            f.write("    data.append(i32 {{ mag: {0}, sign: {1} }});\n".format(abs(int(val)), str(val < 0).lower()))
        f.write(
            "    TensorTrait::new(shape.span(), data.span())\n" +
            "}\n"
        )

with open(os.path.join('./orion_dnn/src', 'generated.cairo'), 'w') as f:
    for param_name in tensors.keys():
        f.write(f"mod {param_name};\n")

In [11]:
import tensorflow_model_optimization as tfmot

# Apply quantization to the layers
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model_tf)

# 'quantize_model' requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 784)               3         
 yer)                                                            
                                                                 
 quant_dense (QuantizeWrapp  (None, 56)                43965     
 erV2)                                                           
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                575       
 pperV2)                                                         
                                                                 
Total params: 44543 (174.00 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 13 (52.00 Byte)
_________________________________________________________________


In [12]:
batch_size = 256
epochs = 3
history = q_aware_model.fit(train_images_tf, train_labels,
                            epochs=epochs,
                            validation_split=0.2)



Epoch 1/3


2024-01-28 12:38:59.657971: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.658006: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659413: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659424: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28 12:38:59.659437: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-01-28

1500/1500 [==============================] - 3s 2ms/step - loss: 0.0963 - accuracy: 0.9720 - val_loss: 0.1078 - val_accuracy: 0.9672
Epoch 2/3
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0788 - accuracy: 0.9758 - val_loss: 0.0958 - val_accuracy: 0.9718
Epoch 3/3
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0657 - accuracy: 0.9806 - val_loss: 0.0903 - val_accuracy: 0.9733


In [13]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [test_images_tf[i:i+1]]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("q_aware_model.tflite", "wb").write(tflite_model)



INFO:tensorflow:Assets written to: /tmp/tmp4j4dhfnv/assets


INFO:tensorflow:Assets written to: /tmp/tmp4j4dhfnv/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-01-28 12:39:13.507507: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-28 12:39:13.507524: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-28 12:39:13.507747: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp4j4dhfnv
2024-01-28 12:39:13.508200: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-28 12:39:13.508209: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp4j4dhfnv
2024-01-28 12:39:13.509435: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 op

ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_input_1:0 

In [9]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="q_aware_model.tflite")
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [29]:
# Select the image for TensorFlow and PyTorch
controlled_input_tf = test_images[34].reshape(1, 28*28)  # Reshape to (1, 784) for DNN
controlled_input_pt = torch.from_numpy(controlled_input_tf).float()

In [30]:
# Create an object with all tensors 
#(an input + all weights and biases)
tensors = {
    "input": controlled_input_pt,
    "fc1_weights": interpreter.get_tensor(1), 
    "fc1_bias": interpreter.get_tensor(2), 
    "fc2_weights": interpreter.get_tensor(4), 
    "fc2_bias": interpreter.get_tensor(5)
}

In [31]:
import os
# Create the directory if it doesn't exist
os.makedirs('./orion_dnn/src/generated', exist_ok=True)

for tensor_name, tensor in tensors.items():
    with open(os.path.join('./orion_dnn/src', 'generated', f"{tensor_name}.cairo"), "w") as f:
        f.write(
            "use core::array::ArrayTrait;\n" +
            "use orion::operators::tensor::{TensorTrait, Tensor, I32Tensor};\n" +
            "use orion::numbers::i32;\n\n" +
            "\nfn {0}() -> Tensor<i32> ".format(tensor_name) + "{\n" +
            "    let mut shape = ArrayTrait::<usize>::new();\n"
        )
        for dim in tensor.shape:
            f.write("    shape.append({0});\n".format(dim))
        f.write(
            "    let mut data = ArrayTrait::<i32>::new();\n"
        )
        for val in np.nditer(tensor.flatten()):
            f.write("    data.append(i32 {{ mag: {0}, sign: {1} }});\n".format(abs(int(val)), str(val < 0).lower()))
        f.write(
            "    TensorTrait::new(shape.span(), data.span())\n" +
            "}\n"
        )
      
with open(os.path.join('./orion_dnn/src', 'generated.cairo'), 'w') as f:
    for param_name in tensors.keys():
        f.write(f"mod {param_name};\n")


In [32]:
! touch orion_dnn/src/nn.cairo


In [33]:
%%writefile orion_dnn/src/nn.cairo
use orion::operators::tensor::core::Tensor;
use orion::numbers::signed_integer::{integer_trait::IntegerTrait, i32::i32};
use orion::operators::nn::{NNTrait, I32NN};

fn fc1(i: Tensor<i32>, w: Tensor<i32>, b: Tensor<i32>) -> Tensor<i32> {
    let x = NNTrait::linear(i, w, b);
    NNTrait::relu(@x)
}

fn fc2(i: Tensor<i32>, w: Tensor<i32>, b: Tensor<i32>) -> Tensor<i32> {
    NNTrait::linear(i, w, b)
}


Overwriting orion_dnn/src/nn.cairo


In [34]:
! touch orion_dnn/src/test.cairo

In [35]:
%%writefile orion_dnn/src/test.cairo
use core::array::SpanTrait;

use mnist_nn::nn::fc1;
use mnist_nn::nn::fc2;
use mnist_nn::generated::input::input;
use mnist_nn::generated::fc1_bias::fc1_bias;
use mnist_nn::generated::fc1_weights::fc1_weights;
use mnist_nn::generated::fc2_bias::fc2_bias;
use mnist_nn::generated::fc2_weights::fc2_weights;

use orion::operators::tensor::I32Tensor;

#[test]
#[available_gas(99999999999999999)]
fn mnist_nn_test() {
    let input = input();
    let fc1_bias = fc1_bias();
    let fc1_weights = fc1_weights();
    let fc2_bias = fc2_bias();
    let fc2_weights = fc2_weights();

    let x = fc1(input, fc1_weights, fc1_bias);
    let x = fc2(x, fc2_weights, fc2_bias);

    let x = *x.argmax(0, Option::None(()), Option::None(())).data.at(0);

    assert(x == 3, 'should predict 2');
}



Overwriting orion_dnn/src/test.cairo


In [36]:
! cd orion_dnn & scarb run test

error: failed to read manifest at: /home/guy1m0/Desktop/ZKML-Benchmark/Milestone 2/MNIST-DNN/Scarb.toml

Caused by:
    No such file or directory (os error 2)


Use Giza-cli to automatically generate cairo script

In [46]:
def to_fixed_point(val, bits):
    return round(val * (2**bits))

def mnist_image_to_fixed_point(data):
    return [to_fixed_point(val.item(), 16) for val in data]


def generate_input_cairo(data):
    values = mnist_image_to_fixed_point(data)
    values = [f"FixedTrait::<FP16x16>::new({val}, {'true' if val < 0 else 'false'})" for val in values]
    return ",\n ".join(values)

In [47]:
controlled_input_pt.shape

torch.Size([1, 784])

In [48]:
input_cairo = generate_input_cairo(controlled_input_pt[0])

In [51]:
with open("mnist_cairo/src/input.cairo", "w") as f:
    f.write("""
use array::{SpanTrait, ArrayTrait};
use orion::operators::tensor::{TensorTrait, FP16x16Tensor, Tensor};
use orion::numbers::{FixedTrait, FP16x16};
fn input() -> Tensor<FP16x16> {
    TensorTrait::<FP16x16>::new(
        array![196].span(),
        array![
    """)
    f.write(input_cairo)
    f.write("""
        ].span()
    )
}
    """)